# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import nexus_tool

After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `file_path` variable to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation and energy demand data.

In [3]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

An excel file containing the fuels and energy data is required. Such file should look something like this:

|Country|Diesel_price_1|Diesel_price_2|El_price_1|El_price_2|
|:---|---|---|---|---|
|Country A|0.17|0.25|0.03|0.05|
|Country B|0.3|0.4|0.1|0.15|
|Country C|0.11|0.18|0.16|0.2|

Change the `file_path` variable to reflect the name and relative location of your data file.

In [4]:
file_path = r'nwsas_data/Fuel_prices.xlsx'
fuel_price = nexus_tool.read_excel(file_path)

## 1. Choosing least-cost energy supply options
### 1.1. Creating the model
To create a model simply create an instance of the `nexus_tool.Model()` class and store it in a variable name. The `nexus_tool.Model()` class requires a dataframe as input data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package.

In [5]:
nwsas = nexus_tool.Model(df)
nwsas.end_year = 2030
nwsas.start_year = 2016
nwsas.discount_rate = 0.05

After creating the model and defining the start and end year, and the discount rate, proceed to merge the `fuel_price` dataframe with the main dataframe of the model. For that run the following block and pass the `on = ` argument with the relevant comon field in both dataframes.

In [6]:
nwsas.df = nwsas.df.merge(fuel_price, on='country')

### 1.2. Creating technologies
The `nexus_tool` model, currently provides three different power generation technology classes to choose from. These are `wind turbines`, `solar PV systems` and a `standard technology` which can be used to simulate technologies as `diesel generators` or `electricity from the grid`. Any amount of technologies can be created and compared in the model. In the following blocks we create a `wind turbine` technology, with a `life` span of 20 years, operation and mantainance costs (`om_cost`) of 1% the capital cost, a capital cost (`capital_cost`) of 1140 USD/kW and a system `efficiency` of 70%. 

The characteristics of the turbine can be specified by changing the power curve (`p_curve`), availability factor (`mu`), rated power (`p_rated`), hub height (`z`), velocity measurement height (`zr`), losses in wind electricity (`0.85`) and wind speed range (`u_arr`). If non of this parameters are passed, the turbine will be created with the default values. To see any of the default values you can call them as `model.technologies['name-of-technology'].p_rated`.

In a similar way, the other technologies can be created calling the right method and passing the required values.

In [7]:
nwsas.create_wind_turbine('Wind power', life=20,
                          om_cost=0.02, capital_cost=1300,
                          efficiency=0.6)

In [8]:
nwsas.create_pv_system('Solar PV', life=15,
                       om_cost=0.01, capital_cost=1140,
                       efficiency=0.7)

In [9]:
nwsas.create_standard_tech('Diesel set', life=10, om_cost=0.1,
                           capital_cost=938, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=0.095, efficiency=0.27, cf = 0.5,
                           emission_factor=2.7, env_cost=0)

In [10]:
nwsas.create_standard_tech('Grid pump', life=10, om_cost=0.1,
                           capital_cost=845, fuel_cost=nwsas.df['Epricelow'],
                           fuel_req=1, efficiency=0.55, cf = 0.8,
                           emission_factor=0.728, env_cost=0)

### 1.3. Getting the LCOE values
To calculate the LCOE values, first the capacity factors, required installed capacity per month and max capacity throughout the year need to be calculated for each technology. For that, run the `get_cf`, `get_installed_capacity` and `get_max_capacity` methods passing a list with the technology names you wish to compute. For example, note that for the `get_cf` method we only pass the wind power and solar PV technology names, as the capacity factors for the grid pumps and diesel sets where passed exogenously when creating the technologies and the `standar_tech` category does not have a defined `get_cf` method. Moreover, to select all technologies a keyword as `'all'`, `'a'` or `everything` can be passed.

After running the aforementioned methods, run the `get_lcoe` method (no argument needs to be passed). This will compute the LCOE values for each technology in each location and store it on a dataframe inside each technology instance.

In [11]:
nwsas.get_cf(['Wind power', 'Solar PV'])
nwsas.get_installed_capacity('all')
nwsas.get_max_capacity('a')
nwsas.get_lcoe()

KeyError: 'annual_el_demand'

### 1.4. Getting the least cost option
To get the least-cost option in each location between all technologies, run the `get_least_cost(technologies = 'all')` method with the `'all'` keyword for the technologies to compare. 

Alternatively, define a dictionary whose keys are the regions and the values a list of technologies to compare withing each region. Pass that dictionary to the `technologies` option of the `get_least_cost` method and specify the `geo_boundary` in which the algorithm should look for the dictionary regions (keys).

In [ ]:
comp_vars = {'Algeria':['Diesel set','Solar PV','Wind power'],
            'Tunisia':['Diesel set','Solar PV','Wind power'],
            'Libya':['Grid pump','Solar PV','Wind power']}
nwsas.get_least_cost(technologies = comp_vars, geo_boundary = 'country')

### 1.5. Getting the pumping cost
Once the least-cost options are identified in each location, then the total pumping cost and pumping cost per m3 of water can be calculated. For it just runb the `get_pumping_cost` and `get_unit_pumping_cost` methods with the `inplace = True` option.

In [ ]:
nwsas.get_pumping_cost(inplace = True)
nwsas.get_unit_pumping_cost(inplace=True)

## 2. Displaying and saving the results
After the calculations are completed, display a summary of results by running the `nwsas.print_summary()` method. If you run the method without any argument, then the summary values will be dispayed for the entire region, under the label of "Glogal". However, if you like to summarize by regions, then pass a list specifing the variables that you want the results to be grouped by, as:

In [ ]:
nwsas.print_summary(['country','province'])

Finally, save the results in .csv format, by specifing an output file name and location (`output_file`) and running the `nwsas.df.to_csv(output_file, index = False)` method.

In [ ]:
output_file = r'nwsas_results/least_cost.csv'
nwsas.df.to_csv(output_file, index = False)